 # 1. Purpose of the project

This project will focus on data collected from stackoverflow. <br>
Using a DL model (BEART) I will try to predict if a question posted on the site is at risk of being closed. <br>
Questions that don't meet the stockoverfull standard are closed because they are irrelevant, poorly written, etc (about 6% of the questions posted). <br>

Main parts:
* Import libraries and load data from database
* Data exploration
* Data cleaning and preparation
* Building our custom BERT model
* Training
* Evaluate the model results
* Conclusions

 # 2. Import libraries and load data from database

### libraries

In [1]:
# libraries

# data manipulation
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import re
import os
from pathlib import Path  
import datetime
import seaborn as sn

# import nlp Tokenizer/tools
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
from transformers import BertTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from tqdm import tqdm
from transformers import BertModel,DistilBertModel
from transformers import AdamW

# Evaluation and data prep
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split

# setup
pd.set_option('display.max_colwidth', None) # want to see all info in a cell

### Loading data

In [2]:
# Loading and printing head
df_stock_over_raw = pd.read_csv("../input/predict-closed-questions-on-stack-overflow/train-sample.csv")

In [3]:
df_stock_over_raw[0:3]

 # 3. Data exploration

### Lets take a look at the diffrent columns

In [4]:
df_stock_over_raw.info()

We have 140,272 data points, lets see how many are closed vs open.

### Lets take a look at the ratio of diffrent status values

In [5]:
df_stock_over_raw.groupby(['OpenStatus']).count()['PostId']/140272

In [6]:
# Lets see the status events by %
stock_over_status_split = df_stock_over_raw.groupby(['OpenStatus']).count()['PostId']/140272
stock_over_status_split.sort_values(ascending=False).plot(kind="bar")
print('Question status %')

We see that about half is open in our sample.

In [7]:
# Lets see if we have more features with a small amount of values
df_stock_over_raw.nunique().sort_values()

In [8]:
# lets see the first 3 headers/body
for i in range(1):
    print('Question number {}: \n\nheader is: \n{}\nbody: \n{} \n'.format(i+1,df_stock_over_raw['Title'].iloc[i],df_stock_over_raw['BodyMarkdown'].iloc[i]))

### Lets take a look at the amount of words we have in the first 95% data points

In [9]:
full_text = df_stock_over_raw['Title'] + df_stock_over_raw['BodyMarkdown']
full_text_count = full_text.str.split().str.len()

In [10]:
full_text_count[full_text_count<full_text_count.quantile(0.95)].hist(bins=10)

So we can easily see that most of our data is below BERT's 512 tolkens limit, thats good!

# 4. Data cleaning and preparation

In [11]:
# Lets start with creating a new column for combining head and body
df_stock_over_raw['text'] = df_stock_over_raw['Title'] + ' ' +df_stock_over_raw['BodyMarkdown']

In [12]:
# looks good!
df_stock_over_raw[0:1][['text','BodyMarkdown','Title']]

In [13]:
# Lets create a new df with only the relavent columns
df_for_bert = df_stock_over_raw[['text','OpenStatus']]

In [14]:
# Lets change the names of our text and lables
df_for_bert.columns = ['text','label']
df_for_bert.columns

In [15]:
# mapping the category name to numbers

labels_status = {'open': 0, 
           'too localized': 1,
           'not a real question': 2,
           'off topic': 3,
           'not constructive':4}

df_for_bert['label'] = df_for_bert['label'].map(labels_status)

### Split train dataset into train, validation and test sets

In [17]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df_for_bert['text'], df_for_bert['label'], 
                                                                    random_state=42, 
                                                                    test_size=0.3, 
                                                                    stratify=df_for_bert['label'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=42, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

### Import BERT Model and BERT Tokenizer

In [18]:
# import BERT model
bert = BertModel.from_pretrained('bert-base-uncased')

# Loading the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Preparing the tokenizers

In [39]:
# setting the maximum tokens possible for BERT 512 or less if GPU space isnt sufficient
max_seq_len = 512

# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

### Casting type to torch tensor

In [40]:
# Train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_y = torch.tensor(train_labels.tolist())
train_mask = torch.tensor(tokens_train['attention_mask'])

# Validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_y = torch.tensor(val_labels.tolist())
val_mask = torch.tensor(tokens_val['attention_mask'])

# Test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_y = torch.tensor(test_labels.tolist())
test_mask = torch.tensor(tokens_test['attention_mask'])

### Creating DataLoaders

In [41]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# 5. Building our custom BERT model

### Class creation

In [42]:
'''
Class BERT_Arch is based on pre-trained model BERT, its designed to output five classes

'''

class BERT_Arch(nn.Module):

    def __init__(self, bert):
        
      
        super(BERT_Arch, self).__init__()

        self.bert = bert 
      
        # dropout layer
        self.dropout = nn.Dropout(0.1)
      
       # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)
      
        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,5)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):
        

        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        
        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)
      
        # apply softmax activation
        x = self.softmax(x)

        return x

### Instantiation the model and transfering to GPU mode

In [43]:
# specify GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)


# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

### Creating the train Class

In [24]:
'''
Function for train the model

Input:  None

Output: 
        avg_loss (Float) - the average loss
        total_preds (List) - List of model predictions
'''

# function to train the model
def train():
  
    model.train()

    total_loss, total_accuracy = 0, 0
  
    # empty list to save model predictions
    total_preds=[]
  
    # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]
 
        sent_id, mask, labels = batch

        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()
  
        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)
    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
  
    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

### Creating the evaluate calss

In [25]:
'''
Function for evaluating the model

Input:  None

Output: 
        avg_loss (Float) - the average loss
        total_preds (List) - List of model predictions
'''

def evaluate():
    
    print("\nEvaluating...")
    
    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0
  
    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(val_dataloader):
    
        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
    

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
        
        # model predictions
        preds = model(sent_id, mask)

        # compute the validation loss between actual and predicted values
        loss = cross_entropy(preds,labels)

        total_loss = total_loss + loss.item()

        preds = preds.detach().cpu().numpy()

        total_preds.append(preds)
    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

# 6. Training

### Freeze BERT Parameters

In [22]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

### Setting up the loss function and number of epochs

In [26]:
# loss function
cross_entropy  = nn.NLLLoss() 

# number of training epochs
epochs = 3

### Training and tarning/validation losses

In [27]:
# set initial loss to infinite
best_valid_loss = float('inf')
model = model.to(device)

# lists for validation and traning loesses
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

# 7. Evaluate the model results

In [46]:
# Push the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

pred_list = []
county = 0


# Get predictions for test data, create a list of predictions to evaluate later
with torch.no_grad():
    model.eval()
    for s,m in zip(test_seq,test_mask):   
        county += 1
        preds = model(s.to(device).reshape(1,512), m.to(device).reshape(1,512))
        preds = preds.detach().cpu().numpy()
        
        if county==10000:
            print('10k')
        if county==20000:
            print('20k')
        
        pred_list.append(preds[0])

In [47]:
# Pick the higest score for prediction
pred_list = np.argmax(pred_list, axis = 1)

In [49]:
# model's performance
print(classification_report(test_y, pred_list))

In [50]:
# confusion matrix

df_cm = pd.crosstab(pred_list,test_y,colnames=['Real value'], rownames=['Predicted value'])
sn.heatmap(df_cm, annot=True, fmt='.5g')

### Saving and loading the models

In [236]:
#save BERT
#torch.save(model.state_dict(),'model_state_v3.pth')

In [44]:
# load BERT
model = BERT_Arch(bert)
model.load_state_dict(torch.load('../input/model-state-v3/model_state_v1.pth'))

# 8. Conclusions

I ran the training loop three times in order to get a better result. <br>
We can see the model definitely "learned" and gets a better score than random prediction.

I see two ways to use the model:

1. Approve any post that comes in and then take only the question that predicted anything that is not 0 (open question), then a human will take a look and decide if the question should be closed. In this case, we can get 12%-33% of type 2/3/4 questions denied and 0% of type 1. The rest will be falsy approved.
2. Approve type 0 prediction automatically, it will mean that we will lose 7% of good questions and give false approval to about 50% of type 2,3,4 of questions and 100% of type 1. This is the easiest way to implment the model, but will probely have to approve the results before.

I think I will keep training the model and maybe change the drop rate/model feature in order to improve my results, especially in category 1.